<b><font color = 'red'><style>
h1 {text-align: center;}</style><h1> Machine Learning Classification</h1> </font> </b>

<b><font color = 'red'> <h4>Churn Prediction </h4></font></b>

In [1]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
# Reading the customer data that we will be using

#data='https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'
#
#!wget $data -o data-week-3.csv  > /dev/null 2>&1

#####> /dev/null 2>&1 redirects all output from wget (both standard output and errors) to "null," effectively suppressing it. This prevents the download messages from appearing in the cell output.



In [3]:
# df1=pd.read_csv('data-week-3.csv')
# df1


In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<b><font color ='red'> Data Preparation </font></b>

In [5]:
df.columns=df.columns.str.lower().str.replace(' ','_')

df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
col_list_object=list(df.dtypes[(df.dtypes=='object')].index)
col_list_object

['customerid',
 'gender',
 'partner',
 'dependents',
 'phoneservice',
 'multiplelines',
 'internetservice',
 'onlinesecurity',
 'onlinebackup',
 'deviceprotection',
 'techsupport',
 'streamingtv',
 'streamingmovies',
 'contract',
 'paperlessbilling',
 'paymentmethod',
 'totalcharges',
 'churn']

In [7]:
for item in col_list_object:
    df[item]=df[item].str.lower().str.replace(' ','_')
    
df

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.5,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-resvb,male,0,yes,yes,24,yes,yes,dsl,yes,...,yes,yes,yes,yes,one_year,yes,mailed_check,84.80,1990.5,no
7039,2234-xaduh,female,0,yes,yes,72,yes,yes,fiber_optic,no,...,yes,no,yes,yes,one_year,yes,credit_card_(automatic),103.20,7362.9,no
7040,4801-jzazl,female,0,yes,yes,11,no,no_phone_service,dsl,yes,...,no,no,no,no,month-to-month,yes,electronic_check,29.60,346.45,no
7041,8361-ltmkd,male,1,yes,no,4,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,yes,mailed_check,74.40,306.6,yes


In [8]:
df.dtypes

customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

In [9]:
#totalcharges column should be numneric ie float or integer but it is object that means that some of the values in the column are string 
#Hence I will try to convert the column to numeric and the error will tell us if that is the case i.e if there are certain values that can no be converted to numeric

pd.to_numeric(df.totalcharges)

ValueError: Unable to parse string "_" at position 488

In [10]:
# that means that the dataset has missing values represented by space and since in our case we replaced the spaces by underscore hence this is this is the case.
# Hence while trying to convert to numeric for errors i will replace those values by NaN using the error parameter

df.totalcharges=pd.to_numeric(df.totalcharges,errors='coerce')

errors='coerce':

The errors parameter tells pd.to_numeric how to handle non-numeric values.
Setting errors='coerce' instructs pandas to replace any non-numeric values with NaN (Not a Number).
For example, if totalcharges has a value 'abc', pd.to_numeric will replace it with NaN instead of throwing an error.


'raise'	Raises an error if any value is non-numeric (default behavior).
'coerce'	Converts non-numeric values to NaN. Useful for handling invalid values without stopping code.
'ignore'	Leaves the values as they are without any conversion.


errors ::	Handles non-numeric values ('raise', 'coerce', or 'ignore').

downcast ::	Attempts to downcast numeric data types ('integer', 'float', 'unsigned').

dtype_backend	:: Specifies the backend for data types (e.g., 'numpy_nullable', 'pyarrow') for specialized data handling.

In [11]:
#Lets check whether the data now has NaN values
df.totalcharges.isnull().sum()

np.int64(11)

In [12]:
# there are 11 values that are null. lets take a look at the customer ids that have these null values
df[df.totalcharges.isnull()][['customerid','totalcharges']]

,customerid,totalcharges
488,4472-lvygi,NaN
753,3115-czmzd,NaN
936,5709-lvoeq,NaN
1082,4367-nuyao,NaN
1340,1371-dwpaz,NaN
3331,7644-omvmy,NaN
3826,3213-vvolg,NaN
4380,2520-sgtta,NaN
5218,2923-arzlg,NaN
6670,4075-wkniu,NaN


In [13]:
#Replace the NaN values by zero
df.totalcharges=df.totalcharges.fillna(0)

In [14]:
#Take a took at our target variable that is churn. ie whether the customer left or not
df.churn

0        no
1        no
2       yes
3        no
4       yes
       ... 
7038     no
7039     no
7040     no
7041    yes
7042     no
Name: churn, Length: 7043, dtype: object

In [15]:
# since this column has Yes and No hence lets change the column to numeric type. since then only we can utilise the column in machine learning
df.churn=(df.churn=='yes').astype(int)

<b><font color='red'> Setting up the validation framework </font></b>

In [16]:
# Now we will split the data in train validation and split.
# We will import the scikit learn library that we will use to train and test and split the data
from sklearn.model_selection import train_test_split

In [17]:
# we can use the ? to see the help/implementation of a function
train_test_split?



Signature:
train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True,
    stratify=None,
)
Docstring:
Split arrays or matrices into random train and test subsets.

Quick utility that wraps input validation,
``next(ShuffleSplit().split(X, y))``, and application to input data
into a single call for splitting (and optionally subsampling) data into a
one-liner.

Read more in the :ref:`User Guide <cross_validation>`.

Parameters
----------
*arrays : sequence of indexables with same length / shape[0]
    Allowed inputs are lists, numpy arrays, scipy-sparse
    matrices or pandas dataframes.

test_size : float or int, default=None
    If float, should be between 0.0 and 1.0 and represent the proportion
    of the dataset to include in the test split. If int, represents the
    absolute number of test samples. If None, the value is set to the
    complement of the train size. If ``train_size`` is also None, it will
    be set to 0.25.

trai

In [18]:
#this will split the sample into the test an train data set  so we have to split the data again in the train and validation data set
df_train_full,df_test=train_test_split(df,test_size=0.2,random_state=1)

In [19]:
#Now we will again split the train data in train and validation also the the size of the test data in now 0.25 since 20% of 100 is 25% of 80
df_train,df_val=train_test_split(df_train_full,test_size=0.25,random_state=1)

len(df_train),len(df_val),len(df_test)

(4225, 1409, 1409)

In [20]:
df_train

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
3897,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,...,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15,0
1980,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,...,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55,0
6302,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65,1
727,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,...,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10,0
5104,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,1309-xgfsn,male,1,yes,yes,52,yes,yes,dsl,no,...,yes,no,yes,yes,one_year,yes,electronic_check,80.85,4079.55,0
6108,4819-hjpiw,male,0,no,no,18,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,no,mailed_check,25.15,476.80,0
1530,3703-vavcl,male,0,yes,yes,2,yes,no,fiber_optic,no,...,yes,yes,no,yes,month-to-month,no,credit_card_(automatic),90.00,190.05,1
3701,3812-lrzir,female,0,yes,yes,27,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,electronic_check,24.50,761.95,0


In [21]:
#Now I can reset the inexes of the dataframe, not that it makes any difference it is just that if the indexes are in increasing order then it makes easier to look for the rows.
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

df_train

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,...,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15,0
1,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,...,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55,0
2,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,...,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65,1
3,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,...,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10,0
4,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4220,1309-xgfsn,male,1,yes,yes,52,yes,yes,dsl,no,...,yes,no,yes,yes,one_year,yes,electronic_check,80.85,4079.55,0
4221,4819-hjpiw,male,0,no,no,18,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,no,mailed_check,25.15,476.80,0
4222,3703-vavcl,male,0,yes,yes,2,yes,no,fiber_optic,no,...,yes,yes,no,yes,month-to-month,no,credit_card_(automatic),90.00,190.05,1
4223,3812-lrzir,female,0,yes,yes,27,yes,yes,no,no_internet_service,...,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,electronic_check,24.50,761.95,0


In [22]:
#Create the target variable arrays
y_train=df_train.churn.values
y_val=df_val.churn.values
y_test=df_test.churn.values

len(y_train),len(y_val),len(y_test)


(4225, 1409, 1409)

In [23]:
# delete the target variable from the train val and test dataframes

del df_train['churn']
del df_val['churn']
del df_test['churn']

df_train


,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges
0,8015-ihcgw,female,0,yes,yes,72,yes,yes,fiber_optic,yes,yes,yes,yes,yes,yes,two_year,yes,electronic_check,115.50,8425.15
1,1960-uycnn,male,0,no,no,10,yes,yes,fiber_optic,no,yes,yes,no,no,yes,month-to-month,yes,electronic_check,95.25,1021.55
2,9250-wypll,female,0,no,no,5,yes,yes,fiber_optic,no,no,no,no,no,no,month-to-month,no,electronic_check,75.55,413.65
3,6786-obwqr,female,0,yes,yes,5,yes,no,fiber_optic,no,no,no,no,yes,no,month-to-month,yes,electronic_check,80.85,356.10
4,1328-euzhc,female,0,yes,no,18,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,20.10,370.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4220,1309-xgfsn,male,1,yes,yes,52,yes,yes,dsl,no,yes,yes,no,yes,yes,one_year,yes,electronic_check,80.85,4079.55
4221,4819-hjpiw,male,0,no,no,18,no,no_phone_service,dsl,no,no,no,no,no,no,month-to-month,no,mailed_check,25.15,476.80
4222,3703-vavcl,male,0,yes,yes,2,yes,no,fiber_optic,no,no,yes,yes,no,yes,month-to-month,no,credit_card_(automatic),90.00,190.05
4223,3812-lrzir,female,0,yes,yes,27,yes,yes,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,electronic_check,24.50,761.95


<b><font color = 'red'> EDA </font></b>

✨Check for missing values
✨Look at the target variable(churn)
✨Look at numerical and categorical variables

In [24]:
df_train_full.reset_index(drop='True',inplace=True)

In [25]:
df_train_full.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [26]:
df_train_full.churn.unique()

array([0, 1])

In [27]:
df_train_full.churn.nunique()

2

In [28]:
df_train_full.churn.value_counts()

churn
0    4113
1    1521
Name: count, dtype: int64

In [29]:
df_train_full.churn.value_counts(normalize=True) #Normalize parameter returns the relative frequencies of the unique values

churn
0    0.730032
1    0.269968
Name: proportion, dtype: float64

In [35]:
#This above value that is 0.269968 is the global churn rate. to calculate it we can directly take the mean of the churn column as well since that is binary variable having values only 0 and 1
gloabl_churn_rate=df_train_full.churn.mean()
round(gloabl_churn_rate,2)

np.float64(0.27)

In [31]:
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [36]:
df_train_full.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [38]:
categorical = [
    'gender',
    'seniorcitizen',
    'partner',
    'dependents',
    'phoneservice',
    'multiplelines',
    'internetservice',
    'onlinesecurity',
    'onlinebackup',
    'deviceprotection',
    'techsupport',
    'streamingtv',
    'streamingmovies',
    'contract',
    'paperlessbilling',
    'paymentmethod',
]

In [33]:
df_train_full[categorical].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

<b><font color ='red'> Feature importance : Churn rate and risk ratio</font></b>

🎇Feature importance analysis (part of EDA) - identifying which features affect our target variable

✨Churn rate
✨Risk ratio
✨Mutual information - later

In [39]:
# now we will take a look at the churn rate for each categorical varible.Lets start with gender.

churn_rate_female=df_train_full[df_train_full.gender=='female'].churn.mean()
churn_rate_female

np.float64(0.27682403433476394)

In [40]:
churn_rate_male=df_train_full[df_train_full.gender=='male'].churn.mean()
churn_rate_male

np.float64(0.2632135306553911)

In [42]:
#Churn rate for parter

churn_rate_partner = df_train_full[df_train_full.partner=='yes'].churn.mean()
churn_rate_partner 

np.float64(0.20503330866025166)

In [ ]:
gloabl_churn_rate-churn_rate_partner # the churn rate for people with partner is 6% less than than the gloable churn rate

np.float64(0.06493474245795922)

In [41]:
churn_rate_nopartner = df_train_full[df_train_full.partner=='no'].churn.mean()
churn_rate_nopartner 

np.float64(0.3298090040927694)

In [ ]:
gloabl_churn_rate-churn_rate_nopartner # i.e it is more than the global churn rate.

# However for male or female the churn rate is approximately the same.

np.float64(-0.05984095297455855)

In [ ]:
# Hence we can say that the partner variable is more important for predicting the churn compared to the gender variable.

#Similarly we can also calculate the risk ratio. Churn rate difference and risk ratio both tell the same thing but in different way. the difference tells us in absolutes while the other one tells us in relative.


In [46]:
risk_ratio_parter=churn_rate_partner/gloabl_churn_rate
risk_ratio_parter

np.float64(0.7594724924338315)

In [47]:
risk_ratio_noparter=churn_rate_nopartner/gloabl_churn_rate
risk_ratio_noparter

np.float64(1.2216593879412643)

In [48]:
#hence we cna say that group/people with no parter are more likely to curn by aproximately 22%
#while group/people with parter are less likely to churn approximately by 25% less

Hence this is how we can decide which variables are important

In [50]:
#but doing this for each categorical variable is lengthy since first we have to know the values of that variable and then we will calculate the churn hence to avoid that
#we can utilise the sql query something like this

# select gender,
# avg(churn) as churn_rate,
# avg(churn)-gloabl_churn_rate as churn_rate_difference,
# avg(churn)/gloabl_churn_rate as risk_ratio
# from data 
# group by gender
